In [10]:
import requests
from urllib.parse import quote

client_id = "SWe77FM8c_EjAm5Y93h5"
client_secret = "mAz_OvLf7T"

def naver_shop_search(query, display=10):
    encoded_query = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded_query}&display={display}&start=1&sort=sim&exclude=used:cbshop:rental"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code}

result = naver_shop_search("영양제", 100)

# if 'items' in result:
#   for item in result['items']:
#     if item['brand'] != '':
#       print(f"브랜드: {item['brand']}")
#       print(f"이름: {item['title']}")
#       print(f"상품ID: {item['productId']}")
#       print(f"링크: {item['link']}")
#       print("-" * 50)
# else:
#   print("Error:", result.get('error', 'Unknown error'))


In [ ]:
!pip install -q chromadb sentence-transformers
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

In [ ]:
# 1. 데이터 전처리 함수
def preprocess_items(items):
    processed = []
    for item in items:
        data = {
            "id": item['productId'],
            "text": f"{item['title']} {item['brand']} {item['maker']} {item['category3']}",  # 임베딩용 텍스트
            "metadata": {
                "price": int(item['lprice']),
                "brand": item['brand'],
                "category": f"{item['category1']}>{item['category2']}>{item['category3']}",
                "link": item['link']
            }
        }
        processed.append(data)
    return processed

# 2. Chroma DB 설정
client = chromadb.PersistentClient(path="./naver_shopping_db")
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# 기존 컬렉션이 있으면 삭제 후 재생성
try:
    client.delete_collection("products")
except:
    pass
collection = client.create_collection(
    name="products",
    embedding_function=embedding_model,  # 반드시 임베딩 함수 재지정
    metadata={"hnsw:space": "cosine"}  # 거리 측정 방식 변경
)

# 3. 데이터 삽입 (API 결과 사용)
if 'items' in result:
    processed_data = preprocess_items(result['items'])

    # 벡터 DB에 문서 추가
    collection.add(
        ids=[item['id'] for item in processed_data],
        documents=[item['text'] for item in processed_data],
        metadatas=[item['metadata'] for item in processed_data]
    )
    print("✅ 성공적으로", len(processed_data), "개 상품 저장됨")
else:
    print("Error:", result.get('error', '데이터 없음'))

# 4. 검색 예시 함수
def search_products(query, n_results=3):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    for idx, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
        print(f"🔍 [{idx}] {doc}")
        print(f"   가격: {meta['price']:,}원 | 브랜드: {meta['brand']}")
        print(f"   카테고리: {meta['category']}")
        print(f"   링크: {meta['link']}")
        print("-"*50)

In [ ]:
search_products("임산부에게 좋은 영양제", 5)

In [ ]:
search_products("어르신들을 위한 영양제", 5)

In [4]:
# Hugging Face 및 Google API 인증
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN') # Hugging Face 액세스 토큰
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Google Custom Search API 키
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID') # Programmable Search Engine ID
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # OpenAI API Key

In [ ]:
# 2. Google Programmable Search Engine 설정
from googleapiclient.discovery import build

def google_search(query, num=3):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]

# Gemma-3-1b-it 모델 초기화
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it", token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)



In [ ]:
!pip install -q langchain langchain-community langchain-core langchain_openai

In [11]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

# 2. 검색 도구 정의 (Function Calling)
@tool
def search_products(query: str, max_results: int = 3) -> list:
    """제품 검색을 위해 Chroma DB를 쿼리하는 도구"""
    results = collection.query(
        query_texts=[query],
        n_results=max_results
    )
    return [{
        "제품": doc,
        "가격": meta['price'],
        "링크": meta['link']
    } for doc, meta in zip(results['documents'][0], results['metadatas'][0])]

# 3. 에이전트 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, openai_api_key=OPENAI_API_KEY)
llm_with_tools = llm.bind_tools([search_products])

system_prompt = """당신은 영양제 전문 약사 어시스턴트입니다. 다음 규칙을 준수하세요:
- 사용자 질문에 대해 search_products 도구를 먼저 사용
- 검색 결과를 바탕으로 정확한 제품 정보 제공
- 가격, 성분, 스펙, 브랜드 비교시 표 형태로 응답
- 모르는 정보는 확대 해석하지 말 것
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

tools = [search_products]
agent = create_openai_tools_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False) # verbose=True 로 변경안해도 됨


In [12]:
# 4. 실행 예시
response = agent_executor.invoke({
    "input": "종종 머리가 아프고 눈이 침침해. 어떤 영양제를 먹으면 좋을지 추천해줘. 주요 성분과 함량도 알려줘."
})
print(response['output'])

다음은 두통과 눈 피로에 도움을 줄 수 있는 영양제 추천 목록입니다. 각 제품의 주요 성분과 가격을 비교해 보세요.

| 제품명 | 가격 | 주요 성분 및 함량 | 링크 |
|--------|------|-------------------|------|
| [1+1]종근당 아임비타 이뮨플러스 샷 2박스 | 23,500원 | 비타민 C, 비타민 D, 아연 등 | [구매하기](https://smartstore.naver.com/main/products/8566190007) |
| 얼라이브 원스데일리 멀티/포맨/우먼 60정 | 23,900원 | 비타민 B군, 비타민 C, 비타민 D, 미네랄 등 | [구매하기](https://smartstore.naver.com/main/products/309612084) |
| 센트룸 실버 포 맨 종합 멀티 비타민 112정 | 34,900원 | 비타민 A, C, D, E, B군, 미네랄 등 | [구매하기](https://smartstore.naver.com/main/products/9244093197) |
| 센트룸 포 맨 더블업 종합 멀티 비타민 112정 | 34,900원 | 비타민 A, C, D, E, B군, 미네랄 등 | [구매하기](https://smartstore.naver.com/main/products/10635135788) |
| 올인원 이뮨 멀티팩 남성 여성 종합비타민 | 38,800원 | 비타민 A, C, D, E, B군, 미네랄 등 | [구매하기](https://smartstore.naver.com/main/products/9057792910) |

각 제품은 다양한 비타민과 미네랄을 포함하고 있어 두통과 눈 피로에 도움을 줄 수 있습니다. 필요에 따라 선택해 보세요.
